### Import

In [19]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from fitparse import FitFile
from os import listdir
from os.path import isfile, join, splitext
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as md

# used by hrv
from datetime import datetime, timedelta


Point the location_string variable to folder containing fitfiles.

In [20]:
location_string = './fitfiles/'
participant_files = []
for f in listdir(location_string):
    if isfile(location_string+f):
        participant_files.append(f)

The following definitions are used to read the messages. The read_hrv_mean should not be used, as it is an incorrect implementation which was created to do some rudimentary checks.

Todo: Implement the correct method for reading hrv messages.

DONE

In [21]:
#brukes for å sammenligne
def speed_corr(speed):
    return speed if isinstance(speed, float) else 0

def power_corr(power):
    return power if isinstance(power, int) else 0

def read_record(rec):
    global curr_timestamp
    global curr_stop
    global curr_active
    if curr_stop == 1 and speed_corr(rec.get_value('speed')) > 7:
        curr_stop = 0
    if ((speed_corr(rec.get_value('speed')) > 0) & (pd.isnull(rec.get_value('power'))) or ((speed_corr(rec.get_value('speed')) > 0) & (power_corr(rec.get_value('power'))> 0))):
        curr_active = 1
    curr_timestamp = rec.get_value('timestamp')
    return (curr_timestamp,
            1, #type = 1
            curr_lap,
            curr_stop, 
            curr_active,
            rec.get_value('speed'), 
            rec.get_value('enhanced_speed'),
            rec.get_value('distance'), 
            rec.get_value('heart_rate'), 
            rec.get_value('accumulated_power'), 
            rec.get_value('power'), 
            rec.get_value('position_lat'), 
            rec.get_value('position_long'), 
            rec.get_value('altitude'), 
            rec.get_value('enhanced_altitude'), 
            rec.get_value('cadence'), 
            rec.get_value('factional_cadence'), 
            rec.get_value('left_right_balnce'), 
            rec.get_value('left_torque_effectiveness'), 
            rec.get_value('right_torque_effectiveness'), 
            rec.get_value('left_pedal_smoothness'), 
            rec.get_value('right_pedal_smoothness'), 
            rec.get_value('temperature'))

def read_lap(rec):
    global curr_timestamp
    global curr_lap
    global curr_stop
    curr_lap += 1
    curr_stop = 1
    curr_timestamp = rec.get_value('timestamp')
    return (curr_timestamp, 
            2, # type = 2
            curr_lap, 
            curr_stop, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            rec.get_value('total_distance'),
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN)

def read_hrv_acc(rec):
    global curr_timestamp
    hrv = []
    for i in range(rec.get_value('time')):
        if rec.get_value('time')[i] != None:
            time_delta = datetime.timedelta(microsecond=rec.get_value('time')[i]*1000)
            curr_timestamp += time_delta
            yield hrv_returner(curr_timestamp)

def hrv_returner(time):
        return (time, 
            3, # type = 2
            curr_lap, 
            curr_stop, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN, 
            np.NaN)

Generator

In [22]:
#skriver om fitgen for å enklere kunne lese og legge til nye messages
def fitgenerator(fitfile):
    for rec in fitfile.get_messages(['record', 'lap', 'hrv']):
        if rec.name=='record':
            yield read_record(rec)
        elif rec.name=='lap':
            yield read_lap(rec)
        else:
            yield read_hrv_acc(rec)

Initializing key variables

In [23]:
fitgen_cols = ['timestamp', 'type', 'lap', 'stop', 'active', 'speed', 'enhanced_speed', 'distance', 'heart_rate', 
               'accumulated_power', 'power', 'position_lat', 'position_long', 
               'altitude', 'enhanced_altitude', 'cadence', 'factional_cadence', 
               'left_right_balnce', 'left_torque_effectiveness', 'right_torque_effectiveness', 
               'left_pedal_smoothness', 'right_pedal_smoothness', 'temperature']
all_dfs = []
all_stops = []
all_hrv = []
all_dStops = []
all_rDfs = []

Convert all messages (defined by generator) to dataframe

In [24]:
for participant_file in participant_files:
    fitfile = FitFile(location_string+participant_file)
    print(participant_file, 'start')
    
    curr_lap = 0
    curr_stop = 0
    curr_active = 0

    fitgen = fitgenerator(fitfile)

    allraw = list(rec for rec in fitfile.get_messages(['record', 'lap', 'hrv']))

    print(participant_file, 'read')
    
    part_df = pd.DataFrame(fitgen, columns=fitgen_cols)
    all_dfs.append(part_df[part_df['type']==1].set_index('timestamp'))
    all_stops.append(part_df[part_df['type']==2].set_index('timestamp'))
    all_hrv.append(part_df[part_df['type']==3].set_index('timestamp'))
    #stopp informasjon.
    all_dStops.append(part_df[part_df['stop']==1].set_index('timestamp'))
    #relativ informasjon. Blir litt feil.
    all_rDfs.append(part_df[part_df['active']==1].set_index('timestamp'))
    
    print(participant_file, 'converted')

50338.FIT start
50338.FIT read


TypeError: object of type 'generator' has no len()